## Unit level model: empirical Bayes (EB) Method

For complex (non-linear) parameters of interest or for non-normally distributed random errors, the EBLUP is not the best predictor and the MSE second-order approximation does not hold. Instead, we will use the Empirical Bayes (EB) method which fits the linear mixed model to the data and use a Monte Carlo (MC) approach to predict the area level complex parameters and their MSE estimates. Thsi approach is implemented by the *EBUnitLevel* class. 

To illustrate the EB method, we will simulate a data following the specification of Molina and Rao (2010). In their paper, they simulated data to analyze the performance of the EB method to estimate area poverty incidences and poverty gaps. 

### Simulated census data

The simulated census data is composed of $N=20,000$ observations from $m=80$ areas with $N_i = 250$ units in each area $i = 1, ..., m,$ with $k_{ij} = 1$. The population are generated using the linear mixed model with random area effects variance $\sigma_u^2 = 0.15^20$ and unit level errors variance $\sigma_e^2 = 0.5^2$. We consider two binary auxiliary variables simulated with $p_{1i} = 0.3+0.5*i/m$ and $p_{2i} = 0.2,$ with $i = 1, ..., m$. The output variable is income. Income is a skewed distribution and to simulated the skewness, we apply an exponential transformation to Y obtained from the linear mixed model discribed above.

In [1]:
#!pip install samplics
%load_ext lab_black
import numpy as np
import pandas as pd

import samplics
from samplics.sampling import Sample
from samplics.sae import EbUnitModel

In [2]:
np.random.seed(123)

# model parameters
scale = 1
sigma2e = 0.5 ** 2
sigma2u = 0.15 ** 2

# Population sizes
N = 20000
nb_areas = 80

# Errors generation
error = np.random.normal(loc=0, scale=(scale ** 2) * (sigma2e ** 0.5), size=N)
area = np.sort(np.random.choice(range(1, nb_areas + 1), N))
areas, Nd = np.unique(area, return_counts=True)
random_effects = np.random.normal(loc=0, scale=sigma2u ** (1 / 2), size=nb_areas)
total_error = np.repeat(random_effects, Nd) + error

# Auxiliary information
p1 = 0.3 + 0.5 * np.linspace(1, nb_areas + 1, nb_areas) / nb_areas
p2 = 0.2
X1 = np.array([]).astype(int)
for i, d in enumerate(areas):
    Xk = np.random.binomial(1, p=p1[i], size=Nd[i])
    X1 = np.append(X1, Xk)
X2 = np.random.binomial(1, p=p2, size=N)
X = np.column_stack((np.ones(N), X1, X2))

beta = np.array([3, 0.03, -0.04])
Y = np.matmul(X, beta) + total_error
income = np.exp(Y)

# Create a dataframe for the population data
census_data = pd.DataFrame(data={"area": area, "X1": X1, "X2": X2, "income": income})

nb_obs = 15
print(f"\nFirst {nb_obs} rows of the population data\n")
census_data.head(nb_obs)


First 15 rows of the population data



,area,X1,X2,income
0,1,1,0,10.910632
1,1,0,0,30.000852
2,1,0,0,20.989920
3,1,0,0,8.579758
4,1,0,1,13.108396
5,1,0,0,41.607121
6,1,0,1,5.202886
7,1,0,0,14.703665
8,1,1,0,35.357877
9,1,0,0,11.812790


Now we re going to select a sample of 50 units in each area. To select the sample, we will use the *Sample* class from *samplics*. 

In [3]:
sample_size = 50
stratum = census_data["area"]
unit_id = census_data.index

sae_sample = Sample(method="srs", stratification=True, with_replacement=False)
sample, _, _ = sae_sample.select(unit_id, sample_size, stratum)
sample_data = census_data[sample == 1]

nb_obs = 15
print(f"\nFirst {nb_obs} rows of the sample data\n")
sample_data.head(nb_obs)


First 15 rows of the sample data



,area,X1,X2,income
0,1,1,0,10.910632
6,1,0,1,5.202886
11,1,1,0,17.907123
13,1,0,0,13.238140
18,1,0,0,30.101651
19,1,0,1,21.234904
24,1,1,0,10.030322
36,1,1,0,18.802240
43,1,0,0,24.275218
45,1,0,0,18.113143


Now that we have simulated a population(census) and selected a sample from it, we are ready to illustrate the steps of the EB method. In summary, we need the sample data with the output variable, the auxiliary variables, and the area variable. And we also need the non-sampled values of the auxiliary variables. In practice, it may challenging to get the out-of-sample auxiliary variables and even more difficult to link the sample to the census in order to partition it into sample vs out-of-sample.

In [4]:
# Sample data
areas = sample_data["area"]
income = sample_data["income"]
Xs = sample_data[["X1", "X2"]]

# Out of sample data
outofsample_data = census_data[sample != 1]
arear = outofsample_data["area"]
Xr = outofsample_data[["X1", "X2"]]

### Fitting the linear mixed model



In [5]:
eb_poverty_reml = EbUnitModel(method="REML", boxcox=0)
eb_poverty_reml.fit(income, Xs, areas, intercept=True)

print(f"\nThe estimated fixed effect using REML: {eb_poverty_reml.fixed_effects}")
print(
    f"\nStandard error of estimated area random effect using REML: {eb_poverty_reml.re_std}"
)
print(
    f"\nStandard error of estimated area random effect using REML: {eb_poverty_reml.error_std}"
)


The estimated fixed effect using REML: [ 2.97923793  0.03629499 -0.02642623]

Standard error of estimated area random effect using REML: 0.15227754550016784

Standard error of estimated area random effect using REML: 0.4972377422416066


In [6]:
eb_poverty_ml = EbUnitModel(method="ML", boxcox=0)
eb_poverty_ml.fit(income, Xs, areas, intercept=True)

print(f"\nThe ML estimated fixed effect using ML is: {eb_poverty_ml.fixed_effects}")
print(
    f"\nStandard error of estimated area random effect using ML: {eb_poverty_ml.re_std}"
)
print(
    f"\nStandard error of estimated area random effect using ML: {eb_poverty_ml.error_std}"
)


The ML estimated fixed effect using ML is: [ 2.97921929  0.03631835 -0.02639921]

Standard error of estimated area random effect using ML: 0.15109887793783536

Standard error of estimated area random effect using ML: 0.4971123962940577


### Prediction of area level poverty indicators

In [7]:
def poverty_gap(y, poverty_line=12):
    return np.mean((y < poverty_line) * (poverty_line - y) / poverty_line)

In [10]:
eb_poverty_reml.predict(Xr, arear, poverty_gap, 2000, poverty_line=12)

import pprint

print("\nREML estimated area level poverty gap using REML\n")
pprint.pprint(eb_poverty_reml.area_est)

Generating the 2000 replicates samples
[===================================================] 102%


REML estimated area level poverty gap using REML

{1: 0.03429308185220425,
 2: 0.029551218425634292,
 3: 0.0530659665483883,
 4: 0.09754209593679143,
 5: 0.008572646652707839,
 6: 0.024978293998052525,
 7: 0.08434668961780194,
 8: 0.016995874115728806,
 9: 0.04343971177323918,
 10: 0.04273946171707999,
 11: 0.04238251249875243,
 12: 0.10019183793789706,
 13: 0.011883209054719629,
 14: 0.02117156835219051,
 15: 0.04251405724639967,
 16: 0.04090014326339838,
 17: 0.021735466712063778,
 18: 0.035135239343597344,
 19: 0.04228391399778987,
 20: 0.037110275727385025,
 21: 0.04310258325243127,
 22: 0.02757933498149177,
 23: 0.013535235723519373,
 24: 0.05447869995841788,
 25: 0.044433007728453086,
 26: 0.016802763125238266,
 27: 0.04510763270554631,
 28: 0.022251452786446177,
 29: 0.0385263171272889,
 30: 0.05368395764814956,
 31: 0.032875150636702086,
 32: 0.02744397536050825,
 33: 0.038627608

In [15]:
eb_poverty_reml.bootstrap_mse(Xr, arear, poverty_gap, 50, poverty_line=12)

import pprint

print("\nREML area level overty gap estimates and their MSE estimates using REML\n")
pprint.pprint(eb_poverty_reml.to_dataframe())

Generating the 50 bootstrap replicate populations
[===================================================] 102%

Fitting and predicting using each of the 50 bootstrap populations
[==================================================] 100%


REML area level overty gap estimates and their MSE estimates using REML

    _area  _estimate  _mse_boot
0       1   0.034293   0.000019
1       2   0.029551   0.000025
2       3   0.053066   0.000359
3       4   0.097542   0.002687
4       5   0.008573   0.000389
..    ...        ...        ...
75     76   0.012565   0.000337
76     77   0.044714   0.000204
77     78   0.028652   0.000038
78     79   0.048149   0.000114
79     80   0.110761   0.004027

[80 rows x 3 columns]


In [13]:
pprint.pprint(eb_poverty_reml.area_mse_boot)

{1: 2.1622883733959723e-05,
 2: 2.7551410137321584e-05,
 3: 0.00030442280596009624,
 4: 0.0023635683592816473,
 5: 0.0003846439390316128,
 6: 6.192892123403974e-05,
 7: 0.0012805284969996705,
 8: 0.00020528983403567843,
 9: 0.00010310372057794199,
 10: 7.664992770004812e-05,
 11: 2.1169151769777854e-05,
 12: 0.002185701088154159,
 13: 0.00030797617489832093,
 14: 0.00011223841127382067,
 15: 5.5042399129355406e-05,
 16: 5.315823652124795e-05,
 17: 5.302033162962843e-05,
 18: 2.939430946154256e-05,
 19: 5.339220848833935e-05,
 20: 2.4555134958981312e-05,
 21: 6.612306463600646e-05,
 22: 2.74529933925964e-05,
 23: 0.00020869235884452133,
 24: 0.0002518189740331753,
 25: 8.838497530484409e-05,
 26: 0.00017266154226812147,
 27: 4.260316442490929e-05,
 28: 0.00010413127735118263,
 29: 2.4371959931922133e-05,
 30: 0.00030979096409367403,
 31: 1.2341303394541672e-05,
 32: 7.06526448452512e-05,
 33: 2.6617740340899764e-05,
 34: 2.3959650610834368e-05,
 35: 5.939963209224146e-05,
 36: 1.2449862